In [5]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import math

In [546]:
## Defining various constants borrowed from the Premium Pricing ipynb notebook
# user inputs 
DURATION = 10
PROTECTION_AMOUNT = 1000000

# predefined values
CURVATURE = 0.05
LEVERAGE_RATIO_FLOOR = 0.05
LEVERAGE_RATIO_CEILING = 3
BUFFER = 0.05
LEVERAGE_RATIO_MIN = LEVERAGE_RATIO_FLOOR - BUFFER
LEVERAGE_RATIO_MAX = LEVERAGE_RATIO_CEILING + BUFFER
MIN_CARAPACE_RISK_PREMIUM = 0.04
UNDERLYING_RISK_PREMIUM_PERCENT = 0.1
PROTECTION_BUYER_APY = 0.17
MINIMUM_STOKEN_UNDERLYING = 1000
MINIMUM_TOTAL_PROTECTION_AMOUNT = 1250

# market driven values
LEVERAGE_RATIO = 0.6

# flags
INSUFFICIENT_STOKEN_UNDERLYING = -1
INSUFFICIENT_TOTAL_PROTECTION_AMOUNT = -2


In [547]:
# Importing functions from carapace repository

def leverage_ratio(
    sToken_underlying=MINIMUM_STOKEN_UNDERLYING,
    total_protection_amount=MINIMUM_TOTAL_PROTECTION_AMOUNT
):
    if(sToken_underlying < MINIMUM_STOKEN_UNDERLYING):
        return INSUFFICIENT_STOKEN_UNDERLYING
    if(total_protection_amount < MINIMUM_TOTAL_PROTECTION_AMOUNT):
        return INSUFFICIENT_TOTAL_PROTECTION_AMOUNT
    return sToken_underlying/total_protection_amount


def carapace_risk_premium(
    duration=DURATION,
    curvature=CURVATURE,
    leverage_ratio=LEVERAGE_RATIO,
    leverage_ratio_min=LEVERAGE_RATIO_MIN,
    leverage_ratio_max=LEVERAGE_RATIO_MAX,
    min_carapace_risk_premium= MIN_CARAPACE_RISK_PREMIUM,
):

    if(leverage_ratio == INSUFFICIENT_STOKEN_UNDERLYING):
        #print("total capital is less than minimum total capital")
        return min_carapace_risk_premium 
    if(leverage_ratio == INSUFFICIENT_TOTAL_PROTECTION_AMOUNT):
        #print("total protection amount is less than minimum total protection amount")
        return min_carapace_risk_premium 
    risk_factor = ((leverage_ratio_max - leverage_ratio) /
                   (leverage_ratio-leverage_ratio_min)) * curvature
    carapace_risk_premium = max((1-pow(math.e, -duration * risk_factor)), min_carapace_risk_premium) 
    return carapace_risk_premium

def underlying_risk_premium(
    duration=DURATION,
    underlying_risk_premium_percent=UNDERLYING_RISK_PREMIUM_PERCENT,
    protection_buyer_apy=PROTECTION_BUYER_APY,
):
    underlying_risk_premium = underlying_risk_premium_percent * duration * protection_buyer_apy
    return underlying_risk_premium

def premium(
    leverage_ratio,
    duration=DURATION,
    curvature=CURVATURE,
    underlying_risk_premium_percent=UNDERLYING_RISK_PREMIUM_PERCENT,
    protection_buyer_apy=PROTECTION_BUYER_APY,
    leverage_ratio_min=LEVERAGE_RATIO_MIN,
    leverage_ratio_max=LEVERAGE_RATIO_MAX,
    min_carapace_risk_premium= MIN_CARAPACE_RISK_PREMIUM,
):

    Carapace_risk_premium = carapace_risk_premium(
        duration=duration,
        curvature=curvature,
        leverage_ratio=leverage_ratio,
        leverage_ratio_min=leverage_ratio_min,
        leverage_ratio_max=leverage_ratio_max,
        min_carapace_risk_premium= min_carapace_risk_premium,
    )

    Underlying_risk_premium = underlying_risk_premium(
        duration=DURATION,
        underlying_risk_premium_percent=underlying_risk_premium_percent,
        protection_buyer_apy=protection_buyer_apy,
    )
    premium = (Carapace_risk_premium + Underlying_risk_premium) 
    return premium

## Basic Example 
### All constants are defined in the way they are defined in the documentation

In [552]:
L = 0.6 # Initial leverage ratio in the junior tranche (chosen randomly)
B = PROTECTION_AMOUNT # B is the buyer demand
Y = PROTECTION_AMOUNT*L # Y is the capital in the junior tranche
assert(L == leverage_ratio(Y,B))

P = premium(L) # Premium percentage initial

# Now add senior pool
X_Y_ratio = 1 # Ratio of X to Y, chosen randomly
X = Y*X_Y_ratio
L_new = leverage_ratio(Y+X,B) # New leverage ratio, referred to as L' in documentation
P_new = premium(L_new) # New premium, referred to as P' in documentation

In [553]:
def senior_pool_premium(B,X,Y,P,P_new,L,L_new):
    return max((B*P_new - (Y*P/L_new)),0)/X

def junior_pool_premium(B,X,Y,P,P_new,L,L_new):
    return min(P/L_new,B*P_new/Y)

In [554]:
s = senior_pool_premium(B,X,Y,P,P_new,L,L_new)
j = junior_pool_premium(B,X,Y,P,P_new,L,L_new)

In [555]:
s,j

(0.3121311726938429, 0.8668231859538017)

## Idea 1

## Extending it to N pools

In [578]:
L_array = [0.6,0.3,0.9,1,0.7] # Array of lending pools (junior tranches)
N = len(L_array) # Number of lending pools
B_array = [PROTECTION_AMOUNT,1.2*PROTECTION_AMOUNT,0.7*PROTECTION_AMOUNT,0.9*PROTECTION_AMOUNT,2*PROTECTION_AMOUNT]
    # array of buyer demand in each lending pool (junior tranches)

assert(len(L_array) == len(B_array))
Y_array = [B_array[i]*L_array[i] for i in range(N)] # array of seller capital in each lending pool

P_array = [premium(L_array[i]) for i in range(N)] # premiums calculated

In [579]:
## Now add senior pool
X = 10*PROTECTION_AMOUNT # A randomly chosen capital amount in the senior pool
L_new_array = [leverage_ratio(Y_array[i]+X/N,B_array[i]) for i in range(N)] # Allocate X/N to each pool and compute
        # new leverage ratios
P_new_array = [premium(L_new_array[i]) for i in range(N)] # New premium arrays 

In [580]:
# Take the average of the senior pool premium over all lending pools
S = sum([senior_pool_premium(B_array[i],X/N,Y_array[i],P_array[i],P_new_array[i],L_array[i],L_new_array[i]) for i in range(N)])/N

In [581]:
S

0.04806423854405022

In [569]:
# Compute the junior tranche premium for each lending pool
J_array = [junior_pool_premium(B_array[i],X/N,Y_array[i],P_array[i],P_new_array[i],L_array[i],L_new_array[i]) for i in range(N)]

In [570]:
J_array

[0.4000722396709854,
 0.5897182524319595,
 0.23079447952948223,
 0.21000000000000002,
 0.5784468147972184]

## Idea 2 

### Representing the senior pool as a higher order Carapace pool

In [582]:
L_array = [0.6,0.3,0.9,1,0.7]# Array of lending pools (junior tranches) 
N = len(L_array) # Number of lending pools
B_array = [PROTECTION_AMOUNT,1.2*PROTECTION_AMOUNT,0.7*PROTECTION_AMOUNT,0.9*PROTECTION_AMOUNT,2*PROTECTION_AMOUNT]
    # array of buyer demand in each lending pool (junior tranches)
assert(len(L_array) == len(B_array))
Y_array = [B_array[i]*L_array[i] for i in range(N)] # array of seller capital in each lending pool

P_array = [premium(L_array[i]) for i in range(N)] # premiums calculated

## Now add senior pool
X = 10*PROTECTION_AMOUNT

In [586]:
# The seller capital is represented as X
# We now calculate the buyer demand

W_array = [(P_array[i])/P_new_array[i] for i in range(N)] # Weight array for each lending pools, currently not used

B_senior = 0
for i in range(N):
    B_senior += max(B_array[i] - Y_array[i],0)   # Add up the exposed capital
    
L_senior = leverage_ratio(X,B_senior) # Compute the premium treating as a Carapace lending pool
S = premium(L_senior) - underlying_risk_premium()# In this case we remove the underling risk premium as we 
# want it entirely on market dynamics, and that is just a constant 

In [587]:
S

0.04000000000000001